# Lab 3

**ПІДСУМОВУВАННЯ ТЕКСТУ ІЗ ЗАСТОСУВАННЯМ МОДЕЛЕЙ HUGGING FACE**

In [44]:
from datasets import load_dataset
import evaluate
from transformers import pipeline
from transformers import (PegasusTokenizer, PegasusForConditionalGeneration,
                          T5Tokenizer, T5ForConditionalGeneration)

In case you need to login

In [ ]:
from huggingface_hub import login
login()

Load ROUTE metric

In [45]:
rouge = evaluate.load("rouge")

## Create Summarizer

In [5]:
model_name = "google/pegasus-cnn_dailymail"

In [6]:
model_name = "facebook/bart-large-cnn"

In [2]:
model_name = "Falconsai/text_summarization"

In [10]:
model_name = "alldaypa/autotrain-nyc_airbnb-71855138766"

In [8]:
model_name = "madushakv/t5_xsum_samsum_billsum_cnn_dailymail"

In [9]:
summarizer = pipeline("summarization", model=model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


## Summarize

In [10]:
with open("backlash_as_new_eu_political_ad_rules_kick_in.txt", 'r') as f:
    text = f.read()

In [14]:
with open("rating_26_years_of_java_changes.txt", 'r') as f:
    text = f.read()

In [11]:
summarizer(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (1303 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Google, Meta and Microsoft have all opted to stop showing political ads in the EU altogether . Politicians on both sides of the aisle said it could be detrimental to democratic debate . The Commission said it is aware of the serious concerns and is continuing talks with Big Tech companies to mitigate the unintended impacts .'}]

## Use Directly

In [ ]:
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [14]:
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Encode input text for the model

In [15]:
inputs = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
inputs

{'input_ids': tensor([[    3,  6934,  3063, 23143,   134,     3,   318,  8767,  1611,  3545,
          2219,  3855,    12,  1172, 13567,   300,   367, 20356,    43,     3,
             7, 16669,     3,     9,  6772,    13, 12334,     6,    38,   779,
          5357,  6979,   323,  1827,  6543,  1446,    13,  9095,    53,     5,
         18409,   277,   497,     8,   973,    56,  1137,     3,     9, 10947,
          1453,    13,   251,   227,    34,     3, 18728,   688,   379,  1163,
             6, 14204,    11,  2803,    12,  4028,     3,     9,  1001,   670,
            30,  1827,  3662,     5, 14984,  7137,    30,   321,  4458,    13,
             8, 23597,   243,    34,   228,    36, 28879,    12, 15053,  5054,
             5,    37,  3527,   243,    34,    19,  2718,    13,     8,  2261,
          3315,    11,    19,  6168,  6927,    28,  2734,  7130,   688,    12,
         21002,     8,    73,    23, 25916, 11737,     5,   486,     8,   842,
            13,     8,  3371,    31,  

Generate summary with custom parameters

In [32]:
summary_ids = model.generate(
    **inputs,
    max_length=30,  # maximum length of the summary
    min_length=10,  # minimum length of the summary
    num_beams=4,  # beam search for better results
    length_penalty=1.0,  # penalize very long summaries
    early_stopping=True,
)
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

'Fresh European Union rules intended to improve transparency around online advertisements have sparked a wave of criticism. Campaigners say the law will cause'

## Experiment

In [33]:
summary_ids = model.generate(
    **inputs,
    do_sample=True,  # enable sampling instead of beam search
    temperature=0.2,  # lower = more focused, higher = more random
    top_k=50,  # limit sampling to top-k words
    max_length=80,
    min_length=20,
)
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

's, Google, Meta and Microsoft have all opted to stop showing political ads in the EU altogether. Campaigners say the law will cause a harmful loss of information after it triggered companies to blackout. Google, Meta and Microsoft have all opted to stop showing political ads in the EU altogether.'

In [37]:
summary_ids = model.generate(
    **inputs,
    do_sample=True,
    temperature=5.0,
    top_k=2,
    top_p=0.7,
    max_length=1000,
    min_length=20,
)
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

'– New European EU law on Transparency and Targeted of Politica. Fresh rules intended to enhance transparency around online ads have prompted a wave of criticism. Politicians on both sides said the law could be detrimental to democratic debate. Campaigners said the law could be detrimental for democracy. The new law on transparency of paid political advertising brings new restrictions and transparency requirement. The new law brings new restrictions and transparencies for paid political ads.'

## Load Dataset

Take first 5 examples for demo

In [39]:
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0", split="train[:5]")
dataset

README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 5
})

In [40]:
generated_summaries = []
reference_summaries = []

for i, sample in enumerate(dataset):
    text = sample["article"]
    reference_summary = sample["highlights"]

    # Generate summary using pipeline
    generated_summary = summarizer(text, min_length=30, max_length=100)[0]["summary_text"]

    # Save for later evaluation
    generated_summaries.append(generated_summary)
    reference_summaries.append(reference_summary)

    print(f"\n=== Example {i+1} ===")
    print("Original article:\n", text[:500], "...")
    print("\nReference summary:\n", reference_summary)
    print("\nGenerated summary:\n", generated_summary)

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 1 ===
Original article:
 LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as s ...

Reference summary:
 Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .

Generated summary:
 Harry Potter star Daniel Radcliffe says he has no plans to fritter cash away on fast cars, drink and celebrity parties . At 18, he will be able to buy a drink in a pub or see the horror film "Hoste

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 2 ===
Original article:
 Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s ...

Reference summary:
 Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

Generated summary:
 Inmates with most severe mental illnesses are incarcerated until they're ready to appear in court . Judge Steven Leifman says arrests

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 3 ===
Original article:
 MINNEAPOLIS, Minnesota (CNN) -- Drivers who were on the Minneapolis bridge when it collapsed told harrowing tales of survival. "The whole bridge from one side of the Mississippi to the other just completely gave way, fell all the way down," survivor Gary Babineau told CNN. "I probably had a 30-, 35-foot free fall. And there's cars in the water, there's cars on fire. The whole bridge is down." He said his back was injured but he determined he could move around. "I realized there was a school bus  ...

Reference summary:
 NEW: "I thought I was going to die," driver says .
Man says pickup truck was folded in half; he just has cut on face .
Driver: "I probably had a 30-, 35-foot free fall"
Minnesota bridge collapsed during rush hour Wednesday .

Generated summary:
 "I probably had a 30-, 35-foot free fall, and there's cars on fire," survivor says . Dr. John Hink jumped into his car and rushed to the scene in 15 minutes . Volunteers, EMTs managed to get

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 4 ===
Original article:
 WASHINGTON (CNN) -- Doctors removed five small polyps from President Bush's colon on Saturday, and "none appeared worrisome," a White House spokesman said. The polyps were removed and sent to the National Naval Medical Center in Bethesda, Maryland, for routine microscopic examination, spokesman Scott Stanzel said. Results are expected in two to three days. All were small, less than a centimeter [half an inch] in diameter, he said. Bush is in good humor, Stanzel said, and will resume his activiti ...

Reference summary:
 Five small polyps found during procedure; "none worrisome," spokesman says .
President reclaims powers transferred to vice president .
Bush undergoes routine colonoscopy at Camp David .

Generated summary:
 NEW: "none appeared worrisome," White House spokesman says . NEW: Bush will resume his activities at Camp David . New: Vice President Dick Cheney assumed presidential power at 9:21 a.m.


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 5 ===
Original article:
 (CNN)  -- The National Football League has indefinitely suspended Atlanta Falcons quarterback Michael Vick without pay, officials with the league said Friday. NFL star Michael Vick is set to appear in court Monday. A judge will have the final say on a plea deal. Earlier, Vick admitted to participating in a dogfighting ring as part of a plea agreement with federal prosecutors in Virginia. "Your admitted conduct was not only illegal, but also cruel and reprehensible. Your team, the NFL, and NFL fa ...

Reference summary:
 NEW: NFL chief, Atlanta Falcons owner critical of Michael Vick's conduct .
NFL suspends Falcons quarterback indefinitely without pay .
Vick admits funding dogfighting operation but says he did not gamble .
Vick due in federal court Monday; future in NFL remains uncertain .

Generated summary:
 NFL commissioner: "Your admitted conduct was not only illegal, but also cruel and reprehensible" Vick admitted to participating in dogfightin

## Evaluate

Compute ROUGE scores using the results

In [46]:
results = rouge.compute(predictions=generated_summaries, references=reference_summaries)
results

{'rouge1': np.float64(0.34465814250497495),
 'rouge2': np.float64(0.15943870816897587),
 'rougeL': np.float64(0.25003677441844835),
 'rougeLsum': np.float64(0.30518590035187765)}